In [3]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
%matplotlib inline

/home/sudeep/.conda/envs/keras_gpu_tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def convert_to_one_hot(Y, C):
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

def load_dataset():
    X_train = np.load('./Data/train_196.npy')
    X_test = np.load('./Data/test_196.npy')
    X_val = np.load('./Data/val_196.npy')
    Y_train = convert_to_one_hot(np.load('./Data/Ltrain_196.npy'), 8)
    Y_test = convert_to_one_hot(np.load('./Data/Ltest_196.npy'), 8)
    Y_val = convert_to_one_hot(np.load('./Data/Lval_196.npy'), 8)
    p1 = np.random.permutation(len(X_train))
    p2 = np.random.permutation(len(X_test))
    p3 = np.random.permutation(len(X_val))
    X_train = X_train[p1, :, :, :]
    X_test = X_test[p2, :, :, :]
    X_val = X_val[p3, :, :, :]
    Y_train = Y_train[p1, :]
    Y_test = Y_test[p2, :]
    Y_val = Y_val[p3, :]
    return X_train, X_test, X_val, Y_train, Y_test, Y_val

In [9]:
def PepperFryModel(input_shape):
   
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)
    
    # layer group1 49*49*32
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (7, 7), strides = (2, 2), name = 'conv1')(X)
    X = BatchNormalization(axis = 3, name = 'bn1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool1')(X)
    
    #layer group2 24*24*64
    X = ZeroPadding2D((2, 2))(X)
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(64, (5, 5), strides = (1, 1), name = 'conv2')(X)
    X = BatchNormalization(axis = 3, name = 'bn2')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool2')(X)
    
    #layer group3 12*12*128
    X = ZeroPadding2D((1, 1))(X)
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(128, (3, 3), strides = (1, 1), name = 'conv3')(X)
    X = BatchNormalization(axis = 3, name = 'bn3')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool3')(X)
    
    #layer group4 12*12*64
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(64, (1, 1), strides = (1, 1), name = 'conv4')(X)
    X = BatchNormalization(axis = 3, name = 'bn4')(X)
    X = Activation('relu')(X)

    #layer group5 6*6*32
    X = ZeroPadding2D((1, 1))(X)
    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv5')(X)
    X = BatchNormalization(axis = 3, name = 'bn5')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((2, 2), name='max_pool5')(X)
    
    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dropout(0.3)(X)
    X = Dense(196, activation='sigmoid', name='fc1')(X)
    X = Dropout(0.3)(X)
    X = Dense(8, activation='softmax', name='fc3')(X)
    
    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='HappyModel')
    
    ### END CODE HERE ###
    
    return model

In [10]:
model = PepperFryModel((196, 196, 3))


In [11]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 196, 196, 3)       0         
_________________________________________________________________
zero_padding2d_9 (ZeroPaddin (None, 202, 202, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 98, 98, 32)        4736      
_________________________________________________________________
bn1 (BatchNormalization)     (None, 98, 98, 32)        128       
_________________________________________________________________
activation_11 (Activation)   (None, 98, 98, 32)        0         
_________________________________________________________________
max_pool1 (MaxPooling2D)     (None, 49, 49, 32)        0         
_________________________________________________________________
zero_padding2d_10 (ZeroPaddi (None, 53, 53, 32)        0         
__________

In [10]:
X_train, X_test, X_val, Y_train, Y_test, Y_val = load_dataset()

Y_train.shape, X_train.shape

((2756, 8), (2756, 196, 196, 3))

In [12]:
model.compile(optimizer = "adam", loss = "categorical_crossentropy", metrics = ["accuracy"])

In [13]:
history = model.fit(x = X_train, y = Y_train, validation_data=(X_val, Y_val), epochs = 5, batch_size = 64)

Train on 2756 samples, validate on 537 samples
Epoch 1/5
2756/2756 [==============================] - 292s 106ms/step - loss: 0.2602 - acc: 0.9245 - val_loss: 0.5621 - val_acc: 0.8101
Epoch 2/5
2756/2756 [==============================] - 310s 112ms/step - loss: 0.1954 - acc: 0.9456 - val_loss: 0.2898 - val_acc: 0.9088
Epoch 3/5
2756/2756 [==============================] - 297s 108ms/step - loss: 0.1637 - acc: 0.9561 - val_loss: 0.3611 - val_acc: 0.8920
Epoch 4/5
2756/2756 [==============================] - 295s 107ms/step - loss: 0.1443 - acc: 0.9634 - val_loss: 0.3577 - val_acc: 0.9032
Epoch 5/5
2756/2756 [==============================] - 294s 107ms/step - loss: 0.1152 - acc: 0.9753 - val_loss: 0.2928 - val_acc: 0.9032


539/539 [==============================] - 25s 46ms/step


[0.30304989218711853, 0.9146567583084106]

In [3]:
from keras.models import load_model
model= load_model('Model_7_epochs_196.h5')

In [12]:
def load_by_name(name_X, name_Y):
    X_train = np.load('./Data/'+name_X+'.npy')
    Y_train = convert_to_one_hot(np.load('./Data/'+name_Y+'.npy'), 8)
    p1 = np.random.permutation(len(X_train))
    X_train = X_train[p1, :, :, :]
    Y_train = Y_train[p1, :]
    return X_train, Y_train
X2, Y2 = load_by_name('X2', "Y2")

In [13]:
X2.shape, Y2.shape

((1916, 196, 196, 3), (1916, 8))

In [11]:
model.fit(x = X1, y = Y1, validation_data=(X_val, Y_val), epochs = 1, batch_size = 128)

Train on 1916 samples, validate on 537 samples
Epoch 1/1
1916/1916 [==============================] - 220s 115ms/step - loss: 0.8300 - acc: 0.7260 - val_loss: 3.8108 - val_acc: 0.3184


In [13]:
model.save('untrained_DropOut.h5')